In [1]:
import torch
from IPython import display
from d2l import torch as d2l
def load_data_fashion_mnist(batch_size,resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root='../data',train=True,transform=trans,download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root='../data',train=False,transform=trans,download=True)
    return (data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers =get_dataloader_workers()),
    data.DataLoader(mnist_test,batch_size,shuffle=False,num_workers=get_dataloader_workers()))

F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
batch_size = 256
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)

In [5]:
#初始化模型参数
num_inputs = 784
num_outputs = 10
W = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b = torch.zeros(num_outputs,requires_grad=True)

In [7]:
#定义softmax操作
x = torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]])
x.sum(0,keepdim=True),x.sum(1,keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [12]:
def softmax(x):
    X_exp =  torch.exp(x)
    partition = X_exp.sum(1,keepdim=True)
    return X_exp /partition

In [13]:
x = torch.normal(0,1,(2,5))
x_prob = softmax(x)
x_prob ,x_prob.sum(1)

(tensor([[0.1397, 0.4685, 0.1552, 0.0955, 0.1412],
         [0.0232, 0.0242, 0.6269, 0.1480, 0.1777]]),
 tensor([1.0000, 1.0000]))

In [14]:
#定义模型
def net(x):
    return softmax(torch.matmul(x.reshape((-1,W.shape[0])),W) +b )

In [16]:
#定义损失函数
y = torch.tensor([0,2])
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],y]

tensor([0.1000, 0.5000])

In [20]:
def cross_entropy(y_hat,y):
    return - torch.log(y_hat[range(len(y_hat)),y])
cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [21]:
def accuracy(y_hat,y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [22]:
accuracy(y_hat,y) / len(y)

0.5

In [24]:
class Accumulator:
    def __init__(self,n):
        self.data = [0.0] * n

    def add(self,*args):
        self.data = [a + float(b) for a,b in zip(self.data,args)]

    def reset(self):
        self.data =[0.0] * len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]


def evaluate_accuracy(net,data_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()
    metric = Accumulator(2)
    with torch.no_grad():
        for x,y in data_iter:
            metric.add(accuracy(net(x),y),y.numel())
        return metric[0] /metric[1]

evaluate_accuracy(net,test_iter)

0.1206

In [25]:
# 训练
def train_epoch_ch3(net,train_iter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.train()

    metric = Accumulator(3)
    for x,y in train_iter:
        y_hat = net(x)
        l = loss(y_hat,y)
        if isinstance(updater,torch.optim.Optimizer):
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            l.sum().backward()
            updater(x.shape[0])
        metric.add(float(l.sum()),accuracy(y_hat,y),y.numer())
    return metric[0]/metric[2],metric[1]/metric[2]